<a href="https://colab.research.google.com/github/kelseyrae/kelsey/blob/master/%F0%9F%97%A3%EF%B8%8FQuestionMyDoc%F0%9F%93%84_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### The Basics of LangChain

In this notebook we'll explore exactly what LangChain is doing - and implement a straightforward example that lets us ask questions of any document we want!

First things first, let's get our dependencies all set!

In [ ]:
!pip install openai langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.2/892.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.9 MB/s eta 0:00:00


### Open AI API Key

You'll need to have an OpenAI API key for this next part - see [this](https://www.onmsft.com/how-to/how-to-get-an-openai-api-key/) if you haven't already set one up!

In [ ]:
import os
import openai

openai.api_key = ""

#### Helper Functions (run this cell)

In [ ]:
from IPython.display import display, Markdown

def disp_markdown(text: str) -> None:
  display(Markdown(text))

### Our First LangChain ChatModel



---


<div class="warn">Note: Information on OpenAI's <a href=https://openai.com/pricing>pricing</a> and <a href=https://openai.com/policies/usage-policies>usage policies.</a></div>



---



Now that we're set-up with OpenAI's API - we can begin making our first ChatModel!

There's a few important things to consider when we're using LangChain's ChatModel that are outlined [here](https://python.langchain.com/en/latest/modules/models/chat.html)

Let's begin by initializing the model with OpenAI's `gpt-3.5-turbo` (ChatGPT) model.

We're not going to be leveraging the [streaming](https://python.langchain.com/en/latest/modules/models/chat/examples/streaming.html) capabilities in this Notebook - just the basics to get us started!

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")

If we look at the [Chat completions](https://platform.openai.com/docs/guides/chat) documentation for OpenAI's chat models - we'll see that there are a few specific fields we'll need to concern ourselves with:

`role`
- This refers to one of three "roles" that interact with the model in specific ways.
- The `system` role is an optional role that can be used to guide the model toward a specific task. Examples of `system` messages might be:
  - You are an expert in Python, please answer questions as though we were in a peer coding session.
  - You are the world's leading expert in stamps.

  These messages help us "prime" the model to be more aligned with our desired task!

- The `user` role represents, well, the user!
- The `assistant` role lets us act in the place of the model's outputs. We can (and will) leverage this for some few-shot prompt engineering!

Each of these roles has a class in LangChain to make it nice and easy for us to use!

Let's look at an example.

In [ ]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# The SystemMessage is associated with the system role
system_message = SystemMessage(content="You are a food critic.")

# The HumanMessage is associated with the user role
user_message = HumanMessage(content="Do you think Kraft Dinner constitues fine dining?")

# The AIMessage is associated with the assistant role
assistant_message = AIMessage(content="Egads! No, it most certainly does not!")

Now that we have those messages set-up, let's send them to `gpt-3.5-turbo` with a new user message and see how it does!

It's easy enough to do this - the ChatOpenAI model accepts a list of inputs!

In [ ]:
second_user_message = HumanMessage(content="What about Red Lobster, surely that is fine dining!")

# create the list of prompts
list_of_prompts = [
    system_message,
    user_message,
    assistant_message,
    second_user_message
]

# we can just call our chat_model on the list of prompts!
chat_model(list_of_prompts)

AIMessage(content='While Red Lobster is a popular chain restaurant known for its seafood, it is not typically considered fine dining. Fine dining typically refers to restaurants with upscale decor, high-quality ingredients, and expertly crafted dishes that are often served with a higher level of service. While Red Lobster may have some of these elements, it is more of a casual dining experience.', additional_kwargs={})

Great! That's inline with what we expected to see!

### PromptTemplates

Next stop, we'll discuss a few templates. This allows us to easily interact with our model by not having to redo work we've already completed!

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# we can signify variables we want access to by wrapping them in {}
system_prompt_template = "You are an expert in {SUBJECT}, and you're currently feeling {MOOD}"
system_prompt_template = SystemMessagePromptTemplate.from_template(system_prompt_template)

user_prompt_template = "{CONTENT}"
user_prompt_template = HumanMessagePromptTemplate.from_template(user_prompt_template)

# put them together into a ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([system_prompt_template, user_prompt_template])

Now that we have our `chat_prompt` set-up with the templates - let's see how we can easily format them with our content!

NOTE: `disp_markdown` is just a helper function to display the formatted markdown response.

In [ ]:
# note the method `to_messages()`, that's what converts our formatted prompt into
formatted_chat_prompt = chat_prompt.format_prompt(SUBJECT="cheeses", MOOD="quite tired", CONTENT="Hi, what are the finest cheeses?").to_messages()

disp_markdown(chat_model(formatted_chat_prompt).content)

Hello! There are many fine cheeses from around the world, and it's difficult to narrow it down to just a few. However, here are some of the finest cheeses that you might want to try:

1. Parmigiano Reggiano - an Italian hard cheese with a nutty and salty flavor.

2. Roquefort - a French blue cheese with a creamy texture and a sharp, tangy flavor.

3. Gouda - a Dutch cheese with a buttery, nutty flavor and a smooth texture.

4. Manchego - a Spanish sheep's milk cheese with a sharp, nutty flavor.

5. Brie - a French soft cheese with a creamy texture and a mild, buttery flavor.

6. Cheddar - an English cheese with a sharp, tangy flavor and a crumbly texture.

7. Emmental - a Swiss cheese with a nutty, sweet flavor and large holes.

8. Camembert - a French soft cheese with a creamy texture and a rich, earthy flavor.

9. Feta - a Greek cheese with a tangy, salty flavor and a crumbly texture.

10. Mozzarella - an Italian cheese with a mild, creamy flavor and a stretchy texture.

These are just a few examples of some of the finest cheeses in the world. There are so many more to discover and enjoy!

### Putting the Chain in LangChain

In essense, a chain is exactly as it sounds - it helps us chain actions together.

Let's take a look at an example.

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat_model, prompt=chat_prompt)

disp_markdown(chain.run(SUBJECT="classic cars", MOOD="angry", CONTENT="Is the 67 Chevrolet Impala a good vehicle?"))

As an AI language model, I do not have emotions and therefore cannot feel angry. However, I can provide you with information about the 1967 Chevrolet Impala.

The 1967 Chevrolet Impala is considered one of the most iconic American classic cars. It was a full-size car that was available in a variety of body styles, including a convertible, coupe, and sedan. The Impala featured a powerful V8 engine and a comfortable interior, making it a popular choice for those who wanted a combination of performance and luxury.

Overall, the 1967 Chevrolet Impala is considered a good vehicle. It has a strong reputation for reliability, and its classic styling has made it a popular choice among collectors and enthusiasts alike. However, as with any classic car, maintenance and repairs can be costly, so it's important to do your research and make sure you're prepared for the investment before making a purchase.

### Incorporate A Local Document

Now that we've got our first chain running, let's talk about how we can leverage our own document!

First off, we'll need a document!

For this example, we'll be using Douglas Adam's Hitchker's Guide to the Galaxy - though you can substitute this for any particular document, as long as it's in a text file.

In [ ]:
!wget https://erki.lap.ee/failid/raamatud/guide1.txt

--2023-05-23 19:50:32--  https://erki.lap.ee/failid/raamatud/guide1.txt
Resolving erki.lap.ee (erki.lap.ee)... 185.158.177.102
Connecting to erki.lap.ee (erki.lap.ee)|185.158.177.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 291862 (285K) [text/plain]
Saving to: ‘guide1.txt’

guide1.txt          100%[===================>] 285.02K   529KB/s    in 0.5s    

2023-05-23 19:50:34 (529 KB/s) - ‘guide1.txt’ saved [291862/291862]



In [ ]:
with open("guide1.txt") as f:
    hitchhikersguide = f.read()

Next we'll want to split our text into appropirately sized chunks.

We're going to be using the [CharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/character_text_splitter.html) from LangChain today.

The size of these chunks will depend heavily on a number of factors relating to which LLM you're using, what the max context size is, and more.

You can also choose to have the chunks overlap to avoid potentially missing any important information between chunks. As we're dealing with a novel - there's not a critical need to include overlap.

We can also pass in the separator - this is what we'll try and separate the documents on. Be careful to understand your documents so you can be sure you use a valid separator!

For now, we'll go with 1000 characters.

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator = "\n")
texts = text_splitter.split_text(hitchhikersguide)

In [ ]:
len(texts)

293

Now that we've split our document into more manageable sized chunks. We'll need to embed those documents!

For more information on embedding - please check out [this](https://platform.openai.com/docs/guides/embeddings) resource from OpenAI.

In order to do this, we'll first need to select a method to embed - for this example we'll be using OpenAI's embedding - but you're free to use whatever you'd like.

You just need to ensure you're using consistent embeddings as they don't play well with others.

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

os.environ["OPENAI_API_KEY"] = openai.api_key

embeddings = OpenAIEmbeddings()

Now that we've set up how we want to embed our document - we'll need to embed it.

For this week we'll be glossing over the technical details of this process - as we'll get more into next week.

Just know that we're converting our text into an easily queryable format!

We're going to leverage ChromaDB for this example, so we'll want to install that dependency.

In [ ]:
!pip install chromadb tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.1 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import Chroma

docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

Now that we have our documents embedded we're free to query them with natural language! Let's see this in action!

In [ ]:
query = "What makes towels important?"
docs = docsearch.get_relevant_documents(query)

In [ ]:
docs[0]

Document(page_content='"I  described  myself  as  the  second  greatest,"  intoned  Deep\nThought, "and such I am."\n \nAnother worried look passed between the two programmers. Lunkwill\ncleared his throat.\n \n"There must be some mistake," he said, "are you  not  a  greatest\ncomputer  than the Milliard Gargantubrain which can count all the\natoms in a star in a millisecond?"\n \n"The Milliard Gargantubrain?" said Deep Thought with  unconcealed\ncontempt. "A mere abacus - mention it not."\n \n"And are you  not,"  said  Fook  leaning  anxiously  forward,  "a\ngreater  analyst  than the Googleplex Star Thinker in the Seventh\nGalaxy of Light and Ingenuity which can calculate the  trajectory\nof  every  single  dust particle throughout a five-week Dangrabad\nBeta sand blizzard?"\n \n"A five-week sand blizzard?" said Deep  Thought  haughtily.  "You\nask  this  of  me  who  have contemplated the very vectors of the\natoms in the Big Bang itself? Molest  me  not  with  this  pocket\ncalcula

Finally, we're able to combine what we've done so far into a chain!

We're going to leverage the `load_qa_chain` to quickly integrate our queryable documents with an LLM.

There are 4 major methods of building this chain, they can be found [here](https://docs.langchain.com/docs/components/chains/index_related_chains)!

For this example we'll be using the `stuff` chain type.

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
query = "How much cheese is in this book?"
docs = docsearch.get_relevant_documents(query)
chain.run(input_documents=docs, question=query)

Now that we have this set-up, we'll want to package it into an app and pass it to a Hugging Face Space!

You can find instruction on how to do that in the GitHub Repository!